# Práctica 4 #  
Grupo: Tralalelo Tralala  
Integrantes: Yuriy Chaban Markevych, Pablo García Fernández, Alejandro Rodríguez Giner y Alonso Campillo Martínez  

# Introducción  
Aquí explicamos el trabajo y la motivación por hacerlo

# Dataset  
Explicamos el dataset elegido, las columnas que vamos a usar y hacemos la limpieza de los datos (previo análisis). Vamos a tratar todos los campos del dataframe como strings. Si en algún futuro necesitamos tratar los datos como números, simplemente los convertiremos.

In [35]:
import sklearn as skl
import pandas as pd
import numpy as np

_prestamos = pd.read_csv('dataset.csv',dtype={ 10 : 'str'})


Ahora vamos a mostrar los 4 primeros y los 4 últimos para asegurar que funciona el dataset a la perfección.

In [36]:
display(_prestamos.head(n=4))
display(_prestamos.tail(n=4))
print(_prestamos.columns.tolist())

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected
4267,4268,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved
4268,4269,1,Graduate,No,9200000,29700000,10,607,17800000,11800000,35700000,12000000,Approved


['loan_id', ' no_of_dependents', ' education', ' self_employed', ' income_annum', ' loan_amount', ' loan_term', ' cibil_score', ' residential_assets_value', ' commercial_assets_value', ' luxury_assets_value', ' bank_asset_value', ' loan_status']


Limpiamos el dataframe para que no tengamos problemas con las claves y las columnas en el futuro.

In [64]:
_prestamos.columns = _prestamos.columns.str.strip()
_prestamos = _prestamos.applymap(lambda x: x.strip() if isinstance(x, str) else x)
display(_prestamos.head(n=1))
lion = _prestamos[_prestamos["education"] == "Not Graduate"]
display(lion)

C:\Users\usuario_local\AppData\Local\Temp\ipykernel_17240\1942747230.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  _prestamos = _prestamos.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
9,10,5,Not Graduate,No,1100000,4300000,10,388,3200000,1400000,3300000,1600000,Rejected
11,12,2,Not Graduate,Yes,6700000,22700000,18,538,15300000,5800000,20400000,6400000,Rejected
12,13,3,Not Graduate,Yes,5000000,11600000,16,311,6400000,9600000,14600000,4300000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4254,4255,0,Not Graduate,Yes,8900000,27700000,6,372,18200000,14900000,33700000,6100000,Rejected
4259,4260,0,Not Graduate,Yes,4500000,11500000,14,509,13400000,2300000,15400000,5900000,Rejected
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected


# Evaluación  
Aquí va la predicción como tal (añadir gráficas si es posible)

# Conclusiones  
Conclusiones sacadas del estudio